1. Deep Learning.
a. Build a DNN with five hidden layers of 100 neurons each, He initialization, and the
ELU activation function.



In [ ]:
import tensorflow as tf

# Build a Deep Neural Network
model = tf.keras.Sequential()

# Input layer
model.add(tf.keras.layers.InputLayer(input_shape=(input_size,)))

# Hidden layers with He initialization and ELU activation
for _ in range(5):
    model.add(tf.keras.layers.Dense(100, activation='elu', kernel_initializer='he_normal'))

# Output layer (adjust the activation function and units based on your problem)
model.add(tf.keras.layers.Dense(output_size, activation='softmax'))

# Display the model summary
model.summary()


b. Using Adam optimization and early stopping, try training it on MNIST but only on
digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You
will need a softmax output layer with five neurons, and as always make sure to save
checkpoints at regular intervals and save the final model so you can reuse it later.

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Load MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Filter digits 0 to 4
train_mask = (train_labels < 5)
test_mask = (test_labels < 5)
train_images, train_labels = train_images[train_mask], train_labels[train_mask]
test_images, test_labels = test_images[test_mask], test_labels[test_mask]

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Build the DNN
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(100, activation='elu', kernel_initializer='he_normal'),
    Dense(100, activation='elu', kernel_initializer='he_normal'),
    Dense(100, activation='elu', kernel_initializer='he_normal'),
    Dense(100, activation='elu', kernel_initializer='he_normal'),
    Dense(100, activation='elu', kernel_initializer='he_normal'),
    Dense(5, activation='softmax')  # Softmax output layer with five neurons
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks for checkpointing and early stopping
checkpoint_cb = ModelCheckpoint("model_checkpoint.h5", save_best_only=True)
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)

# Train the model
history = model.fit(train_images, train_labels, epochs=50, validation_split=0.2,
                    callbacks=[checkpoint_cb, early_stopping_cb])

# Save the final model
model.save("final_model.h5")



11490434/11490434 [==============================] - 167s 15us/step


Epoch 1/50


765/765 [==============================] - 5s 6ms/step - loss: 0.1183 - accuracy: 0.9618 - val_loss: 0.0555 - val_accuracy: 0.9825
Epoch 2/50
  1/765 [..............................] - ETA: 3s - loss: 0.0103 - accuracy: 1.0000

C:\Users\Rushabh.Patel2\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


765/765 [==============================] - 3s 4ms/step - loss: 0.0579 - accuracy: 0.9812 - val_loss: 0.0657 - val_accuracy: 0.9806
Epoch 3/50
765/765 [==============================] - 3s 4ms/step - loss: 0.0412 - accuracy: 0.9879 - val_loss: 0.0529 - val_accuracy: 0.9853
Epoch 4/50
765/765 [==============================] - 3s 4ms/step - loss: 0.0373 - accuracy: 0.9880 - val_loss: 0.0476 - val_accuracy: 0.9842
Epoch 5/50
765/765 [==============================] - 3s 4ms/step - loss: 0.0283 - accuracy: 0.9904 - val_loss: 0.0358 - val_accuracy: 0.9899
Epoch 6/50
765/765 [==============================] - 3s 4ms/step - loss: 0.0213 - accuracy: 0.9931 - val_loss: 0.0582 - val_accuracy: 0.9845
Epoch 7/50
765/765 [==============================] - 3s 4ms/step - loss: 0.0207 - accuracy: 0.9939 - val_loss: 0.0446 - val_accuracy: 0.9897
Epoch 8/50
765/765 [==============================] - 3s 4ms/step - loss: 0.0163 - accuracy: 0.9952 - val_loss: 0.0414 - val_accuracy: 0.9899
Epoch 9/50
765/76

c. Tune the hyperparameters using cross-validation and see what precision you can
achieve.

In [10]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Load MNIST dataset
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Filter for digits 0 to 4
mask_train = (y_train_full <= 4)
mask_test = (y_test <= 4)

X_train, y_train = X_train_full[mask_train], y_train_full[mask_train]
X_test, y_test = X_test[mask_test], y_test[mask_test]

# Flatten and normalize the pixel values
X_train = X_train.reshape(X_train.shape[0], -1) / 255.0
X_test = X_test.reshape(X_test.shape[0], -1) / 255.0

# One-hot encode the target labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=5)

# Define the DNN model function
def create_dnn_model():
    model = Sequential()
    model.add(Dense(100, activation='elu', kernel_initializer='he_normal', input_shape=(784,)))
    model.add(Dense(100, activation='elu', kernel_initializer='he_normal'))
    model.add(Dense(100, activation='elu', kernel_initializer='he_normal'))
    model.add(Dense(100, activation='elu', kernel_initializer='he_normal'))
    model.add(Dense(100, activation='elu', kernel_initializer='he_normal'))
    model.add(Dense(5, activation='softmax'))
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and compile the DNN model
dnn_model = create_dnn_model()

# Set up callbacks for early stopping and model checkpoint
early_stopping_cb = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint_cb = ModelCheckpoint("dnn_model.h5", save_best_only=True)

# Train the model on the training set
history = dnn_model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stopping_cb, model_checkpoint_cb])

# Evaluate the model on the test set
test_loss, test_accuracy = dnn_model.evaluate(X_test, y_test)

# Hyperparameter tuning using cross-validation
param_grid = {'batch_size': [32, 64, 128], 'epochs': [50, 100, 150]}
model = KerasClassifier(build_fn=create_dnn_model, verbose=0)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='precision_weighted', verbose=2)
grid_result = grid_search.fit(X_train, y_train)

# Print the best parameters and corresponding precision score
print("Best Parameters: ", grid_result.best_params_)
print("Best Precision Score: ", grid_result.best_score_)


ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'